In [2]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import (AutoARIMA, 
                                  DynamicOptimizedTheta as DOT,
                                  AutoETS,
                                  AutoTheta,
                                  WindowAverage,                              
                                  CrostonClassic as Croston,
                                  ADIDA,
                                  IMAPA, 
                                  )
from multiprocessing import cpu_count
from datetime import timedelta

from sklearn.metrics import mean_absolute_percentage_error
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from prophet import Prophet
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from pmdarima.arima import auto_arima

import warnings
%matplotlib inline
warnings.filterwarnings("ignore")

c:\Users\andre\AppData\Local\Programs\Python\Python310\lib\site-packages\statsforecast\core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


The notebook is segmented into distinct sections, which we explain below. Our primary focus lies on functions, followed by execution chunks.

### Load Data

In [ ]:
id_list = pd.read_csv('../out/id_selection/id_selected.csv')
id_list.set_index('unique_id', inplace=True)
id_list = id_list.index.unique()
id_list

In [2]:
weekly_selid_clean = pd.read_csv('../out/sales_files/weekly_sales_selected_loop_without_test_outliers_2019-2022.csv')
weekly_selid_clean = weekly_selid_clean.set_index('unique_id')
weekly_selid_clean = weekly_selid_clean[['ds', 'y']]
weekly_selid_clean['ds'] = pd.to_datetime(weekly_selid_clean['ds'])
weekly_selid_clean

,ds,y
unique_id,,
00610101,2019-01-07,97.0
00610101,2019-01-14,95.0
00610101,2019-01-21,124.0
00610101,2019-01-28,143.0
00610101,2019-02-04,216.0
...,...,...
999777,2022-11-28,29.0
999777,2022-12-05,29.0
999777,2022-12-12,29.0


Log Transformation


In [ ]:
log_weekly_selid = weekly_selid_clean.copy()
log_weekly_selid['y'] = np.log1p(log_weekly_selid.y)
log_weekly_selid

### Evaluation functions

* Define error metrics.
* Function for applicating error metrics to cross-validation dataframe.
* Set of functions to obtain summary of error metrics by levels and models.



In [5]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred-y_true))

def wape_mtly(y_true, y_pred): 
    wape2 = abs((y_true.sum() - y_pred.sum())) / y_true.sum() * 100
    if np.isinf(wape2):
        wape2 = np.nan
    return wape2


In [6]:
from datasetsforecast.losses import rmse

def evaluate(cv_df):
    cv_df.reset_index(inplace=True)
    cv_df.set_index('unique_id', inplace = True)

    columns = cv_df.drop(columns=['ds', 'cutoff']).columns.tolist()
    exp_cv_df = cv_df.copy()  

    for column in columns:
        exp_cv_df[column] = np.expm1(cv_df[column])

    metrics = pd.DataFrame(columns=['id','model','rmse', 'mae', 'wape_m'])
    models = cv_df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()

    for index in exp_cv_df.index.unique():
        cv_one_id = exp_cv_df[exp_cv_df.index == index]

        for model in models:
            cv_one_id_model = cv_one_id[['ds','y',model]]
            rmse_one_id_one_model = cv_one_id_model.groupby(cv_one_id_model['ds'].dt.month).apply(lambda x: rmse(x['y'], x[model]))
            mean_rmse = rmse_one_id_one_model.mean()
            mae_one_id_one_model = cv_one_id_model.groupby(cv_one_id_model['ds'].dt.month).apply(lambda x: mae(x['y'], x[model]))
            mean_mae = mae_one_id_one_model.mean()
            wape2_one_id_one_model = cv_one_id_model.groupby(cv_one_id_model['ds'].dt.month).apply(lambda x: wape_mtly(x['y'], x[model]))
            mean_wape2 = np.nanmean(wape2_one_id_one_model)
            row = {'id': index, 
                'model': model, 
                'rmse': round(mean_rmse),  
                'mae': round(mean_mae,1),
                'wape_m': round(mean_wape2,1)
                }
            
            metrics = pd.concat([metrics, pd.DataFrame(row, index=[0])], ignore_index=True)
    
    return metrics

#### Metrics summary:

In this section we defined the following procedures:
* Select best model within each product, using MAE.
* Create a table with descriptive metrics of each product
* Classify WAPE of each product in levels and merge with descriptive metrics table.
* Create final summary table: Group products in WAPE levels and compute grouped metrics
* Count number of products per model
* Compute global WAPE


In [5]:
def select_best_models(df, metric):
    min_metric_models = df.groupby('id')['mae'].idxmin()
    best_models = df.loc[min_metric_models, ['id', 'model', 'mae', metric]]
    column_title = f'{metric}_level'
    best_models[column_title] = np.where(best_models[metric] == 0, '0%',
                           np.where((best_models[metric] > 0) & (best_models[metric] <= 10), '1% - 10%',
                           np.where((best_models[metric] > 10) & (best_models[metric] <= 20), '11% - 20%',
                           np.where((best_models[metric] > 20) & (best_models[metric] <= 30), '21% - 30%',
                           np.where((best_models[metric] > 30) & (best_models[metric] <= 40), '31% - 40%',
                           np.where((best_models[metric] > 40) & (best_models[metric] <= 50), '41% - 50%',
                           np.where((best_models[metric] > 50) & (best_models[metric] <= 60), '51% - 60%',
                           np.where((best_models[metric] > 60) & (best_models[metric] <= 70), '61% - 70%',
                           np.where((best_models[metric] > 70) & (best_models[metric] <= 80), '71% - 80%',
                           np.where((best_models[metric] > 80) & (best_models[metric] <= 90), '81% - 90%',
                           np.where((best_models[metric] > 90) & (best_models[metric] <= 100), '91% - 100%',
                           np.where((best_models[metric] > 100), '>100%',                      
                           'inf'))))))))))))
    best_models.rename(columns={'id': 'unique_id'}, inplace=True)
    best_models.set_index('unique_id', inplace=True)
    return best_models

In [6]:
def create_descriptive_metrics_table():
  summary_weekly_sales = pd.DataFrame()
  summary_weekly_sales['mean_last_8_weeks'] = weekly_selid_clean.groupby('unique_id')['y'].rolling(window=8, min_periods=1).mean().groupby('unique_id').last()
  summary_weekly_sales['historic_mean'] = weekly_selid_clean.groupby('unique_id')['y'].mean().round(decimals=2)
  summary_weekly_sales['weeks_since_first_sale'] = weekly_selid_clean.groupby('unique_id')['ds'].count()
  summary_weekly_sales['sales_2022'] = weekly_selid_clean[(weekly_selid_clean['ds'] >= '2022-01-01')].groupby('unique_id').sum()
  return summary_weekly_sales

In [7]:
def compute_error_levels(summary, best_models, metric_level):
  error_levels = summary.merge(best_models, left_index=True, right_index=True)
  error_levels.reset_index(inplace=True)
  error_levels.set_index(metric_level, inplace=True)
  return error_levels

In [8]:
def create_error_level_summary(weekly_error_levels, metric):
        metric = f'{metric}_level'
        total_sales_2022 = weekly_error_levels['sales_2022'].sum()
        weekly_results = pd.DataFrame({metric: ['0%','1% - 10%','11% - 20%','21% - 30%','31% - 40%', '41% - 50%', '51% - 60%', '61% - 70%','71% - 80%',
                '81% - 90%', '91% - 100%', '>100%']})
        weekly_results.set_index(metric, inplace=True)
        weekly_results['unique_id'] = weekly_error_levels.groupby(metric)['unique_id'].count()
        weekly_results['mean_last_8_weeks'] = weekly_error_levels.groupby(metric)['mean_last_8_weeks'].mean().round(decimals=2)
        weekly_results['historic_mean'] = weekly_error_levels.groupby(metric)['historic_mean'].mean().round(decimals=2)
        weekly_results['weeks_since_first_sale'] = weekly_error_levels.groupby(metric)['weeks_since_first_sale'].mean().round(decimals=2)
        weekly_results['sales_2022'] = weekly_error_levels.groupby(metric)['sales_2022'].sum().round(decimals=2)
        weekly_results['% sales_2022'] = round((weekly_error_levels.groupby(metric)['sales_2022'].sum().round(decimals=2)/total_sales_2022)*100, 2)
        #weekly_results.to_csv(f'../out/summary_tables/weekly_results_{metric}.csv')
        return weekly_results


In [9]:
def summarize_errors(metrics_wkl, metric):
  best_weekly_models = select_best_models(metrics_wkl, metric)
  best_weekly_models.sort_values(by=metric)
  descriptive_metrics = create_descriptive_metrics_table()
  error_levels = compute_error_levels(descriptive_metrics, best_weekly_models, f'{metric}_level')
  results_summary = create_error_level_summary(error_levels, metric)
  models_table = error_levels.groupby('model')['unique_id'].count().sort_values(ascending=False)
  global_error = error_levels[metric].mean()

  return results_summary, models_table, global_error


### Exploratory Modeling

In this section we defined the following functions:
* Data transformations
* Apply 4 models (Mean, Random Walk, Window Average, AutoArima) using cross-validation.
* Prophet tuning with grid search for hyperparameters optimization 
* Apply prophet with cross-validation and best parameters



In [12]:
def m1_initial_transformations(df):
    df = df.reset_index()
    df = df.set_index('ds')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

In [13]:
def m1_models(df):
        split_cv = TimeSeriesSplit(n_splits=4, test_size=4*2*1, gap=0) 
        cross_validation = pd.DataFrame()
        
        for train_idx, val_idx in split_cv.split(df):
                
                train = df.iloc[train_idx]
                test = df.iloc[val_idx]
                test['cutoff'] = test.index.min() - timedelta(weeks=1)
                
                # Mean
                mean_model = train['y'].mean()
                test['mean'] = mean_model

                # Random Walk
                test['random_walk'] = train.y.tail(n=1).values[0]
                
                # Window Average
                train['MA4'] = train['y'].rolling(window = 4).mean()
                test['window_average'] = train['MA4'].tail(n=1).values[0]

                # ARIMA
                target = train['y']
                model = auto_arima(target)
                test['auto_arima'] = model.predict(n_periods = len(test))

                cross_validation = pd.concat([cross_validation, test])

        return cross_validation

In [14]:
def m1_tuning_hyperparameters_prophet(df, days):
    new_df = pd.DataFrame({'ds': df['ds'],'y':df['y']}).reset_index()
        
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative']
    }

    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  

    for params in all_params:
        m = Prophet(**params, growth='flat') 
        m.add_country_holidays(country_name='ES')
        m.fit(new_df) 
        df_cv = cross_validation(m, initial=f'{days} days', period='56 days', horizon = '56 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    best_params = all_params[np.argmin(rmses)]
    return best_params

In [15]:
def m1_apply_prophet(p_df, best_params, days, m1_cv):

    auto_model = Prophet(changepoint_prior_scale=best_params['changepoint_prior_scale'], 
                        seasonality_prior_scale=best_params['seasonality_prior_scale'], 
                        seasonality_mode=best_params['seasonality_mode'], growth='flat')

    auto_model.add_country_holidays(country_name='ES')
    
    auto_model.fit(p_df)
    
    auto_model_cv = cross_validation(auto_model, initial=f'{days} days', period='56 days', horizon = '56 days', parallel="processes")
    auto_model_cv.set_index('ds', inplace=True)
    m1_cv['prophet'] = auto_model_cv['yhat']
    
    return m1_cv, auto_model

In order to apply these 5 models the portfolio, we created a loop that iterates each function across each product.
Finally, we:
* Evaluate 5 models
* Summarize metrics

In [ ]:
m1_global_crossvalidation = pd.DataFrame()

for id in log_weekly_selid.index.unique():

  df = log_weekly_selid.loc[id]
  one_id_df = m1_initial_transformations(df)
  crossvalidation = m1_models(one_id_df)
  days = (crossvalidation.index.min() - df['ds'].min()).days - 30
  best_params = m1_tuning_hyperparameters_prophet(df, days)
  crossvalidation, prophet_model = m1_apply_prophet(df, best_params, days, crossvalidation)
  global_crossvalidation = pd.concat([global_crossvalidation, crossvalidation])

m1_global_crossvalidation

We load a pre-computed CSV files, as the execution of the loop takes several hours or even days to complete.

In [16]:
#m1_global_crossvalidation.to_csv('../out/cv/crossvalidation_m1_models.csv')
m1_global_crossvalidation = pd.read_csv('../out/cv/crossvalidation_m1_models.csv')
m1_global_crossvalidation.set_index('ds', inplace=True)
m1_global_crossvalidation.index = pd.to_datetime(m1_global_crossvalidation.index)

In [18]:
#m1_global_metrics = evaluate(m1_global_crossvalidation)
#m1_global_metrics.to_csv('../out/metrics/m1_global_metrics.csv')
m1_global_metrics = pd.read_csv('../out/metrics/m1_global_metrics.csv')
#m1_global_metrics.head(50)

In [39]:
m1_results_summary, m1_models_table, m1_global_wape_m = summarize_errors(m1_global_metrics, 'wape_m')
m1_results_summary

In [ ]:
m1_models_table

In [ ]:
m1_global_wape_m

### Final Modeling

In this section we include:
* Models training
* Models evaluation using cross-validation
* Forecasting



In [ ]:
log_sf = StatsForecast(
    models = [AutoARIMA(season_length = 52), AutoETS(season_length=52), DOT(season_length = 52), AutoTheta(season_length = 52), ADIDA(), Croston(), IMAPA(), WindowAverage(window_size=4)],
    freq = 'W',
    n_jobs = cpu_count()-1,
    fallback_model= WindowAverage(window_size=4)
)

log_sf.fit(log_weekly_selid)

A pre-trained model for this project can be loaded, as the training process takes several hours or even days to complete.

In [10]:
#joblib.dump(log_sf, '../models/weekly_selected_log_model.pkl')
log_sf = joblib.load('../models/weekly_selected_log_model.pkl')

Each series was subjected to 4-blocks cross-validation with an 8-week extension for the testing period.

In [ ]:
log_crossvalidation_df = log_sf.cross_validation(
    df = log_weekly_selid,
    h = 8, 
    step_size=8, 
    n_windows=4
)

log_crossvalidation_df
#log_crossvalidation_df.to_csv('../out/cv/crossvalidation_weekly_8_models_2019-2022_log.csv')

In [ ]:
log_crossvalidation_df = pd.read_csv('../out/cv/crossvalidation_weekly_8_models_2019-2022_log.csv')
log_crossvalidation_df.set_index('unique_id', inplace=True)
log_crossvalidation_df['ds'] = pd.to_datetime(log_crossvalidation_df['ds'])
log_crossvalidation_df['cutoff'] = pd.to_datetime(log_crossvalidation_df['cutoff'])
log_crossvalidation_df

In [ ]:
#f_global_metrics = evaluate(log_crossvalidation_df)
#f_global_metrics.to_csv('../out/metrics/metrics_selected_weekly_exp_wape2monthly.csv')
f_global_metrics = pd.read_csv('../out/metrics/metrics_selected_weekly_exp_wape2monthly.csv')
f_global_metrics

In [ ]:
f_results_summary, f_models_table, f_global_wape_m = summarize_errors(f_global_metrics, 'wape_m')
f_results_summary

In [ ]:
f_models_table

In [ ]:
f_global_wape_m

* We calculated the sales forecasting that corresponds to one year (52 weeks)
* We convert the logarithmic forecast values back to their exponential form.
* We chart the forecasts, delineating products by their respective models.

In [22]:
#log_forecast = log_sf.forecast(h=52, level=[90])
#log_forecast.to_csv('../out/forecast/weekly_log_52_8.8.23.csv')

log_forecast = pd.read_csv('../out/forecast/weekly_log_52_8.8.23.csv')
log_forecast.set_index('unique_id', inplace=True)
log_forecast['ds'] = pd.to_datetime(log_forecast['ds'])
#log_forecast

In [24]:
columns = log_forecast.drop(columns=['ds']).columns.tolist()
f_exp_forecast = log_forecast.copy() 

for column in columns:
    f_exp_forecast[column] = round(np.expm1(log_forecast[column]))

f_exp_forecast

In [ ]:
best_models = select_best_models(f_global_metrics, 'wape_m')
summary = create_descriptive_metrics_table()
merged_bmodels_summary = summary.merge(best_models, left_index=True, right_index=True).sort_values(by='historic_mean', ascending=False)

models = log_crossvalidation_df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()
model_ids = {}

for model in models:
    ids = merged_bmodels_summary[merged_bmodels_summary['model'] == model].index
    model_ids[model] = ids

model_ids

In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['AutoARIMA'], unique_ids=model_ids['AutoARIMA'].values, level=[90], plot_random = False)

In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['AutoETS'], unique_ids=model_ids['AutoETS'].values, level=[90], plot_random = False)

In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['DynamicOptimizedTheta'], unique_ids=model_ids['DynamicOptimizedTheta'].values, level=[90], plot_random = False)

In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['AutoTheta'], unique_ids=model_ids['AutoTheta'].values, level=[90], plot_random = False)

In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['ADIDA'], unique_ids=model_ids['ADIDA'].values, level=[90], plot_random = False)


In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['CrostonClassic'], unique_ids=model_ids['CrostonClassic'].values, level=[90], plot_random = False)

In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['WindowAverage'], unique_ids=model_ids['WindowAverage'].values, level=[90], plot_random = False)

In [ ]:
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['AutoETS'], unique_ids=['9134235'], level=[90])
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['AutoARIMA'], unique_ids=['9130190'], level=[90], plot_random = False)
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['DynamicOptimizedTheta'], unique_ids=['29961001'], level=[90])
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['AutoTheta'], unique_ids=['20010301'], level=[90], plot_random = False)
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['WindowAverage'], unique_ids=['29925064'], level=[90])
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['ADIDA'], unique_ids=['20735201NM'], level=[90])
log_sf.plot(weekly_selid_clean, f_exp_forecast, models=['CrostonClassic'], unique_ids=['134617'], level=[90])

### One Time Series Example

In [ ]:
example_id = '130335'
example_id_best_model = 'AutoARIMA'
df_example_id = weekly_selid_clean.loc[example_id]
log_example_id = log_weekly_selid.loc[example_id]
StatsForecast.plot(log_example_id)

### 1st Group of Models

In [ ]:
ex_df = m1_initial_transformations(log_example_id)
ex_crossvalidation = m1_models(ex_df)
days = (ex_crossvalidation.index.min() - log_example_id['ds'].min()).days - 30
ex_best_params = m1_tuning_hyperparameters_prophet(log_example_id, days)
ex_crossvalidation, prophet_model = m1_apply_prophet(log_example_id, ex_best_params, days, ex_crossvalidation)


In [ ]:
ex_future = prophet_model.make_future_dataframe(periods=52, freq='w')
ex_forecast = prophet_model.predict(ex_future)
prophet_model.plot(ex_forecast)


In [ ]:
prophet_model.plot_components(ex_forecast)

### 2nd Group of Models


In [10]:
one_sf = StatsForecast(
    models = [AutoARIMA(season_length = 52), AutoETS(season_length=52), DOT(season_length = 52), AutoTheta(season_length = 52), ADIDA(), Croston(), IMAPA(), WindowAverage(window_size=4)],
    freq = 'W',
    n_jobs = cpu_count()-1,
    fallback_model= WindowAverage(window_size=4)
)

one_sf.fit(log_example_id)

one_log_cv_df = one_sf.cross_validation(
    df = log_example_id,
    h = 8, 
    step_size=8, 
    n_windows=4 
)

one_log_forecast = one_sf.forecast(h=52, level=[90])

In [ ]:
one_sf.plot(log_example_id, one_log_forecast)

In [ ]:
cutoff = one_log_cv_df['cutoff'].unique()

for k in range(len(cutoff)): 
    cv = one_log_cv_df[one_log_cv_df['cutoff'] == cutoff[k]]
    StatsForecast.plot(log_example_id, cv.loc[:, cv.columns != 'cutoff'], models=[example_id_best_model])

In [ ]:
one_serie_metrics = evaluate(one_log_cv_df)
one_serie_metrics